# Rag Deployment Development

In [1]:
from model_library.cards.architectures import LOCAL_AI_ARCHITECTURE
from model_library.cards.models import LOCAL_AI_MODEL_CARD
from model_library.cards.deployments import LOCAL_AI_DEPLOYMENT_CARD
from model_library.cards.model_deployment_templates import LOCAL_AI_DEPLOYMENT_TEMPLATE_CARD
from model_library.models import *   
from model_library import ARCHITECTURES, MODELS, DEPLOYMENTS, MODEL_DEPLOYMENT_TEMPLATES

/home/atomkins/miniconda3/envs/kalavai/lib/python3.9/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_card" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/atomkins/miniconda3/envs/kalavai/lib/python3.9/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_deployment_template" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
user = UserInformation(
    id="2",
    API_key= "XXXX-XXXX-XXXX-XXXX",
    namespace = "adam",
)



In [18]:
# Define the RAG Data Structure
from uuid import uuid4

# Assuming ArchitectureCard, ModelCard, DeploymentCard, and ModelDeploymentTemplateCard are defined as per your instructions

# Architecture Specific Details
architecture = ArchitectureCard(
    id=str(uuid4()),
    deployment_yaml="kalavai_rag_deployment.yaml",
    name="Kalavai Rag Tool",
    description="RAG tool deployment architecture.",
    tags={},
    health_endpoint="/health"  # or specify if applicable
)


# Deployment Specific Details
deployment_card = DeploymentCard(
    id=str(uuid4()),
    params={"cpu_limits": "2", "memory_limits": "4Gi", "cpu_requests": "2", "memory_requests": "4Gi"}
)

# Model Deployment Template (combining all the above)
model_deployment_template_card = ModelDeploymentTemplateCard(
    id=str(uuid4()),
    name="RAG Tool Deployment",
    description="Deployment template for RAG tool with Python 3.8-slim container.",
    model_card=model_card,
    deployment_card=deployment_card,
    benchmarks=[],
    params={"namespace": "tools", "replicas": 1, "deployment_name": "rag-tool-test"}
)

In [22]:
from model_library.cards.model_deployment_templates import RAG_DEPLOYMENT_TEMPLATE_CARD

model_deployment = ModelDeploymentCard(
    user_information=user,
    model_deployment_template=RAG_DEPLOYMENT_TEMPLATE_CARD,
)

deployment_config = model_deployment.extract_deployment_config()

ImportError: cannot import name 'RAG_DEPLOYMENT_TEMPLATE_CARD' from 'model_library.cards.model_deployment_templates' (/home/adam/kalavai/model_library/model_library/cards/model_deployment_templates.py)

In [20]:
file_path = "rag_deployment_example.yaml"  # Replace with your desired file path

# Writing the string to a file
with open(file_path, 'w') as file:
    file.write(deployment_config)

print(f"YAML string written to {file_path}")



YAML string written to rag_deployment_example.yaml


In [16]:
import requests
import json

def deploy_generic_model(config):
    # Read the configuration from the file

    # Prepare the request data
    data = {
        "config": config
    }

    # Define the URL and headers
    url = "http://0.0.0.0:8000/v1/deploy_generic_model"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }

    # Make the POST request
    response = requests.post(url, headers=headers, data=json.dumps(data))

    # Check the response
    if response.status_code == 200:
        print("Deployment successful.")
        print(response.json())
    else:
        print(f"Deployment failed with status code {response.status_code}.")
        print(response.text)




In [17]:
deploy_generic_model(deployment_config)

Deployment successful.
{'successful': ['[[{\'api_version\': \'v1\',\n \'kind\': \'Service\',\n \'metadata\': {\'annotations\': None,\n              \'creation_timestamp\': datetime.datetime(2024, 2, 5, 16, 37, 41, tzinfo=tzutc()),\n              \'deletion_grace_period_seconds\': None,\n              \'deletion_timestamp\': None,\n              \'finalizers\': None,\n              \'generate_name\': None,\n              \'generation\': None,\n              \'labels\': {\'app\': \'rag-tool-app\'},\n              \'managed_fields\': [{\'api_version\': \'v1\',\n                                  \'fields_type\': \'FieldsV1\',\n                                  \'fields_v1\': {\'f:metadata\': {\'f:labels\': {\'.\': {},\n                                                                            \'f:app\': {}}},\n                                                \'f:spec\': {\'f:externalTrafficPolicy\': {},\n                                                           \'f:internalTrafficPolicy\'

In [2]:
from model_library.client import ModelLibraryClient

In [9]:
model_library_url = "http://127.0.0.1:8001"
model_library_api_key = None

library = ModelLibraryClient(model_library_url, model_library_api_key)



In [28]:
# Todo streamline this
rag_template = {t["id"]:t for t in library.get_all()["ModelDeploymentTemplateCard"]}["rag_tool"]

KeyError: 'rag_tool'

In [27]:

from model_library.models import *
namespace = "adam"
API_key = "XXX"

# Create a User
user = UserInformation(id=namespace, namespace=namespace, API_key=API_key)

# Create a Config
config = library.create_kubernetes_deployment(
    template=rag_template,
    user=user.dict()
)

<Response [200]>


In [ ]:

# Deploy Model
kube_watcher = KubeWatcherClient(kube_watcher_url, kube_watcher_api_key)
res = kube_watcher.deploy_generic_model({"config":st.session_state.config})